# This is the the comparison of the Test Results for Audio Reasoner using our data

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("output_audio_reasoner.csv")
# df['AnsReason'] = ref['Answer'].astype(str) + " " + ref['Reasoning'].astype(str)
df['Reference'] = "<REASONING>\n" + df['Reasoning'].astype(str) + "</REASONING>\n<ANSWER>\n" + df['Answer'].astype(str) + "\n</ANSWER>"

df['Predicted'] = df['ModelResponse'].apply(lambda x: '\n'.join(x.split('\n')[1:]))

df.head(2)

,file_id,Question,Answer,Reasoning,ModelResponse,Reference,Predicted
0,mfe8OzzArGc_chunk29_data1_task3,What cause of depression does this show?,This patient shows causes of depression relate...,The text raises a question about feeling at ri...,<THINK>\n<REASONING>\nThe audio presents a con...,<REASONING>\nThe text raises a question about ...,<REASONING>\nThe audio presents a conversation...
1,jm6PG989Q_0_chunk78_data1_task1,Does the patient suffer from loneliness?,"Yes, the patient suffers from loneliness.",The patient expresses feelings of helplessness...,<THINK>\n<REASONING>\nThe question asks whethe...,<REASONING>\nThe patient expresses feelings of...,<REASONING>\nThe question asks whether the pat...


In [ ]:
# print(df.iloc[0,1],df.iloc[0,2],df.iloc[0,3],df.iloc[0,4],sep="\n\n")
print(df.iloc[0,5],df.iloc[0,6],sep="\n\n")

## COMET scores

In [12]:
from comet import download_model, load_from_checkpoint

model_path = download_model("Unbabel/wmt22-comet-da")
model = load_from_checkpoint(model_path)

/data/amey_2311cs10/anaconda3/envs/condapy312/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 5 files: 100%|████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 55043.36it/s]
Lightning automatically upgraded your loaded checkpoint from v1.8.3.post1 to v2.5.1.post0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../.cache/huggingface/hub/models--Unbabel--wmt22-comet-da/snapshots/2760a223ac957f30acfb18c8aa649b01cf1d75f2/checkpoints/model.ckpt`
Encoder model frozen.
/data/amey_2311cs10/anaconda3/envs/condapy312/lib/python3.12/site-packages/pytorch_lightning/core/saving.py:195: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']


In [13]:
# Prepare data in COMET format
comet_reason_data = [
    {
        "src": "",  # Source left empty
        "mt": df.loc[i, "Predicted"],
        "ref": df.loc[i, "Reference"]
    }
    for i in range(len(df))
]

In [ ]:
comet_reason_data

In [16]:
model_output = model.predict(comet_reason_data, batch_size=8, gpus=1)  # Set gpus=0 if no GPU

# Individual scores
comet_scores = model_output.scores

# Average score
avg_comet = sum(comet_scores) / len(comet_scores)

print(f"Average COMET score: {avg_comet:.4f}")

Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA H100 PCIe') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
Predicting DataLoader 0: 100%|████████████████████████████████████████████████████████| 225/225 [00:37<00:00,  6.00it/s]


Average COMET score: 0.6721


# Chunked SBERT + Max Matching + Average

In [3]:
import nltk
nltk.download('punkt')
import pandas as pd
from sentence_transformers import SentenceTransformer, util

[nltk_data] Downloading package punkt to
[nltk_data]     /data/amey_2311cs10/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/data/amey_2311cs10/anaconda3/envs/condapy312/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# Load SBERT model
model = SentenceTransformer('all-MiniLM-L6-v2')  # or 'all-mpnet-base-v2' for better quality

In [5]:
# Fallback if nltk not available
def safe_sent_tokenize(text):
    try:
        import nltk
        return nltk.sent_tokenize(text)
    except:
        # Basic sentence split fallback
        return [s.strip() for s in text.split('.') if s.strip()]

In [7]:
# SBERT similarity function with fallback tokenization
def sbert_similarity(text1, text2):
    sents1 = safe_sent_tokenize(text1)
    sents2 = safe_sent_tokenize(text2)

    emb1 = model.encode(sents1, convert_to_tensor=True)
    emb2 = model.encode(sents2, convert_to_tensor=True)

    sim_matrix = util.pytorch_cos_sim(emb1, emb2)
    ref_to_pred = sim_matrix.max(dim=1).values.mean().item()
    pred_to_ref = sim_matrix.max(dim=0).values.mean().item()

    return (ref_to_pred + pred_to_ref) / 2

# Assuming df has 'reference' and 'predicted' columns
similarities = []

for idx, row in df.iterrows():
    try:
        score = sbert_similarity(row['Reference'], row['Predicted'])
    except Exception as e:
        print(f"Error at row {idx}: {e}")
        score = None
    similarities.append(score)



# Add to DataFrame
df['sbert_similarity'] = similarities

# Compute and print mean similarity (excluding None)
valid_scores = [s for s in similarities if s is not None]
mean_score = sum(valid_scores) / len(valid_scores)
print(f"Mean SBERT Semantic Similarity: {mean_score:.4f}")

Mean SBERT Semantic Similarity: 0.6222
